In [ ]:
# %load raw_bad_contents.py
"""
Created on Fri Jun 14 23:40:31 2024

@author: ASUS
"""
import hashlib
import json ##数据解析
import os ##文件路径操作
import time ##时间格式化与线程休眠
import pandas  ##数据导出
import requests ##发送网络请求
from lxml import etree  ##数据解析

In [ ]:
##定义函数，用于发送网络请求，并捕获异常
def send_get(url,headers,params):
    while 1:
        try:
            ##发送请求
            response = requests.get(
                url,##请求地址
                headers=headers,##请求头
                params=params,##请求参数
                timeout=(4,5),##请求超时时间
            )
            ##判断是否被防采集
            if '像机器人程序' in response.text:
                print('像机器人程序')
                continue
            ##响应正文
            return response.text
        except Exception as e:
            print(f'some error:{e}')
            time.sleep(1)

In [ ]:
##定义采集帖子内容函数
def crawl_group_url(gurl):

    ##提取到小组链接里的小组号
    gid = gurl.split("group/")[-1].split("/")[0]

    ##遍历页码采集
    for page in range(0,20):

        ##改变url中值的start
        start = page * 25

        ##发送请求，首页链接
        url = f'https://www.douban.com/group/699153/discussion?start={start}&type=essence'
        
        response = send_get(url,headers,{})

        ##构建document对象
        document = etree.HTML(response)

        ##获取讨论列表
        alltr = document.xpath(".//table[@class='olt']//tr")[1:]

        print(page,len(alltr))

        ##遍历讨论列表，通过xpath解析到讨论列表里的标题，链接，作者等内容
        for itr in alltr:

            saveitme = {}
            saveitme["帖子id"] = "".join(itr.xpath("./td[1]/a/@href")).strip().split("/")[-2]
            saveitme["帖子标题"] = "".join(itr.xpath("./td[1]/a/@title")).strip()
            saveitme["帖子链接"] = "".join(itr.xpath("./td[1]/a/@href")).strip()
            saveitme["帖子作者"] = "".join(itr.xpath("./td[2]//text()")).strip()
            saveitme["帖子作者id"] = "".join(itr.xpath("./td[2]/a/@href")).strip().split("/")[-2]
            saveitme["帖子回复数"] = "".join(itr.xpath("./td[3]//text()")).strip()
            saveitme["帖子最新回复时间"] = "".join(itr.xpath("./td[4]//text()")).strip()

            print(page,saveitme)

            ##访问讨论的详情页，获取讨论正文与评论内容
            detail_respons = send_get(saveitme["帖子链接"],headers_no_cookie,{})
            detail_document = etree.HTML(detail_respons)

            saveitme["发帖时间"] = "".join(detail_document.xpath(".//span[@class='create-time color-green']/text()"))
            saveitme["发帖ip属地"] = "".join(detail_document.xpath(".//span[@class='ip-location']/text()"))
            saveitme["发帖内容"] = "".join(detail_document.xpath(".//div[@class='rich-content topic-richtext']//text()"))

            ##保存帖子信息到名为f的txt中
            with open("post.txt",'a',encoding='utf-8') as f:
                f.write(json.dumps(saveitme))
                f.write('\n')

In [ ]:
if  __name__ == "__main__":

    ##携带cookie请求头
    user_cookie = 'bid=hKqovRvCqCo; _pk_id.100001.8cb4=f1b4692e519846e7.1713102061.; douban-fav-remind=1; viewed="25900948_35287359_21632268_3062632"; __gads=ID=03e8662162097fb0:T=1713428793:RT=1716016202:S=ALNI_MY42qNzZ_0eV_MGHhqFjG_EFb9aFA; __gpi=UID=00000df0cd1a249f:T=1713428793:RT=1716016202:S=ALNI_MbE22YhnCxSRpAOaaGEJFYbttCiAg; __eoi=ID=477005d6f632b98c:T=1713428793:RT=1716016202:S=AA-AfjbMD3E426P0S_UrAcRBVePQ; dbcl2="264423676:Uf0YEvQHGMQ"; push_noty_num=0; push_doumail_num=0; __utmv=30149280.26442; __yadk_uid=gWdcmhGDIm8eTWc62mnjGoT7yCTN0mXk; ap_v=0,6.0; ck=0vkF; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1718119618%2C%22https%3A%2F%2Fwww.bing.com%2F%22%5D; _pk_ses.100001.8cb4=1; __utma=30149280.1580798998.1713102062.1718110041.1718119618.18; __utmc=30149280; __utmz=30149280.1718119618.18.14.utmcsr=bing|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; frodotk_db="11fb39fc1d79046435f1adf4586b0d2c"; __utmb=30149280.24.8.1718119862829'
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0',
        "Referer": 'https://www.douban.com/group/699153/discussion?start=50&type=essence',
        "Cookie": user_cookie,
        "Accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        "Host": 'www.douban.com'
    }

    ##没携带cookie的请求头
    headers_no_cookie = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0',
        "Referer": 'https://www.douban.com/group/699153/discussion?start=50&type=essence',
        "Accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        "Host": 'www.douban.com'
    }

    ##小组链接
    group_url = f'https://www.douban.com/group/699153/'
    ##通过之前定义的函数采集小组帖子详细信息
    crawl_group_url(group_url)

    ##导出到excel
    with open("post.txt",'r',encoding='utf-8') as f:
        lines = [json.loads(i.strip()) for i in f.readlines()]
        ##打开f文件，遍历每一行，解析
    df = pandas.DataFrame(lines) ##将解析后的数据存入df
    df.to_excel("失败帖子表.xlsx",index=False) ##将fd存到excel中
    os.remove("post.txt") ##删除f文件